In [1]:
# Singapore Air Quality and Weather Data Cleaning
# This notebook cleans and processes:
# 1. Pollutant data (2016-2024) -> single output file with DAILY aggregation
# 2. Weather data (air temperature, humidity, wind speed) -> yearly output files with DAILY aggregation

import pandas as pd
import numpy as np
import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("Libraries imported successfully!")

# ============================================================================
# 1. CLEAN POLLUTANT DATA - DAILY AGGREGATION
# ============================================================================

def clean_pollutant_data(years=range(2016, 2025), aggregate='day'):
    """
    Clean and combine pollutant data from all years
    Output columns: Region, Date, pm25, pm10, o3, no2, so2, co, aqi

    Parameters:
      - years: iterable of years to process
      - aggregate: 'none' | 'hour' | 'day' -> how to aggregate timestamps
        'day' (recommended): One record per region per day with daily averages
        'hour': One record per region per hour with hourly averages
        'none': Keep all raw timestamps (may have multiple per day)
    """
    all_pollutant_data = []
    
    for year in years:
        file_path = f'pollutant/pollutants_{year}.csv'
        
        if not os.path.exists(file_path):
            print(f"Warning: {file_path} not found, skipping...")
            continue
        
        print(f"Processing pollutant data for {year}...")
        
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Parse datetime
        df['Date'] = pd.to_datetime(df['date'], errors='coerce')
        
        # Build cleaned dataframe with numeric conversions
        df_clean = pd.DataFrame({
            'Region': df['region'].str.title(),
            'Date': df['Date'],
            'pm25': pd.to_numeric(df['pm25_twenty_four_hourly'], errors='coerce'),
            'pm10': pd.to_numeric(df['pm10_twenty_four_hourly'], errors='coerce'),
            'o3': pd.to_numeric(df['o3_eight_hour_max'], errors='coerce'),
            'no2': pd.to_numeric(df['no2_one_hour_max'], errors='coerce'),
            'so2': pd.to_numeric(df['so2_twenty_four_hourly'], errors='coerce'),
            'co': pd.to_numeric(df['co_eight_hour_max'], errors='coerce'),
            'aqi': pd.to_numeric(df['psi_twenty_four_hourly'], errors='coerce')
        })
        
        # Remove rows with all missing values in numeric columns
        numeric_columns = ['pm25', 'pm10', 'o3', 'no2', 'so2', 'co', 'aqi']
        df_clean = df_clean.dropna(subset=numeric_columns, how='all')
        
        # Apply aggregation based on parameter
        if aggregate == 'day':
            # Floor to day level (remove time component)
            df_clean['Date'] = df_clean['Date'].dt.floor('D')
            # Group by Date (day) and Region, take mean of all numeric columns
            grouped = df_clean.groupby(['Date', 'Region'])[numeric_columns].mean().reset_index()
            # Round numeric columns to 2 decimal places
            for col in numeric_columns:
                grouped[col] = grouped[col].round(2)
            # Format date as YYYY-MM-DD (no time component needed for daily data)
            grouped['Date'] = grouped['Date'].dt.strftime('%Y-%m-%d')
            all_pollutant_data.append(grouped)
            print(f"  - Processed {len(grouped)} daily records from {year}")
            
        elif aggregate == 'hour':
            # Floor to hour level
            df_clean['Date'] = df_clean['Date'].dt.floor('H')
            # Group by Date (hour) and Region
            grouped = df_clean.groupby(['Date', 'Region'])[numeric_columns].mean().reset_index()
            # Round numeric columns
            for col in numeric_columns:
                grouped[col] = grouped[col].round(2)
            # Format with hour precision
            grouped['Date'] = grouped['Date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
            all_pollutant_data.append(grouped)
            print(f"  - Processed {len(grouped)} hourly records from {year}")
            
        else:  # aggregate == 'none'
            # Keep raw timestamps, just format them
            df_clean['Date'] = df_clean['Date'].dt.strftime('%Y-%m-%dT%H:%M:%S')
            # Round numeric columns
            for col in numeric_columns:
                df_clean[col] = df_clean[col].round(2)
            all_pollutant_data.append(df_clean)
            print(f"  - Processed {len(df_clean)} raw records from {year}")
    
    # Combine all years
    if all_pollutant_data:
        combined_df = pd.concat(all_pollutant_data, ignore_index=True)
        
        # Sort by datetime and region
        combined_df = combined_df.sort_values(['Date', 'Region']).reset_index(drop=True)
        
        # Save to CSV
        output_file = 'pollutant_data.csv'
        combined_df.to_csv(output_file, index=False)
        
        print(f"\n✓ Pollutant data cleaned and saved to '{output_file}'")
        print(f"  Total records: {len(combined_df):,}")
        print(f"  Date range: {combined_df['Date'].min()} to {combined_df['Date'].max()}")
        print(f"  Regions: {sorted(combined_df['Region'].unique())}")
        
        # Show records per region to verify daily aggregation
        if aggregate == 'day':
            print(f"\n  Records per region:")
            records_per_region = combined_df.groupby('Region').size()
            for region, count in records_per_region.items():
                print(f"    {region}: {count:,} days")
        
        return combined_df
    else:
        print("No pollutant data files found!")
        return None

# Execute pollutant data cleaning with DAILY aggregation
print("\n" + "="*80)
print("CLEANING POLLUTANT DATA")
print("="*80)
pollutant_df = clean_pollutant_data(aggregate='day')

# Display sample of cleaned pollutant data
if pollutant_df is not None:
    print("\n" + "="*80)
    print("POLLUTANT DATA SAMPLE")
    print("="*80)
    print("\nFirst 10 records:")
    print(pollutant_df.head(10))
    print("\nData types:")
    print(pollutant_df.dtypes)
    print("\nSummary statistics:")
    print(pollutant_df.describe())
    print("\nMissing values:")
    missing = pollutant_df.isnull().sum()
    missing_pct = (missing / len(pollutant_df) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Percentage': missing_pct
    })
    print(missing_df[missing_df['Missing Count'] > 0])

# ============================================================================
# 2. CLEAN WEATHER DATA - DAILY AGGREGATION
# ============================================================================

def map_station_to_region(station_name):
    """
    Map station names to regions based on Singapore geography
    """
    station_name = station_name.lower()
    
    # North region stations
    north_keywords = ['ang mo kio', 'sembawang', 'yishun', 'woodlands', 'admiralty', 
                      'seletar', 'pulau ubin', 'mandai']
    # South region stations
    south_keywords = ['sentosa', 'marina', 'tanjong', 'harbourfront', 'alexandra',
                      'bukit merah', 'telok blangah', 'keppel']
    # East region stations
    east_keywords = ['changi', 'pasir ris', 'tampines', 'bedok', 'east coast',
                     'simei', 'paya lebar', 'kim chuan']
    # West region stations
    west_keywords = ['jurong', 'boon lay', 'tuas', 'choa chu kang', 'bukit batok',
                     'bukit panjang', 'clementi', 'banyan', 'nanyang']
    # Central region stations
    central_keywords = ['newton', 'bishan', 'serangoon', 'toa payoh', 'novena',
                        'bukit timah', 'orchard', 'tai seng']
    
    for keyword in north_keywords:
        if keyword in station_name:
            return 'North'
    for keyword in south_keywords:
        if keyword in station_name:
            return 'South'
    for keyword in east_keywords:
        if keyword in station_name:
            return 'East'
    for keyword in west_keywords:
        if keyword in station_name:
            return 'West'
    for keyword in central_keywords:
        if keyword in station_name:
            return 'Central'
    
    # Default to Central if no match
    return 'Central'

def clean_weather_data_by_year(year):
    """
    Clean and combine weather data (temperature, humidity, wind speed) for a specific year
    Output columns: Country, Region, Date, Temperature, RelativeHumidity, WindSpeed
    Daily aggregation: One record per region per day
    """
    print(f"\nProcessing weather data for {year}...")
    
    # File paths
    temp_file = f'weather/air_temperature/air_temperature_{year}.csv'
    humidity_file = f'weather/humidity/humidity_{year}.csv'
    wind_file = f'weather/wind_speed/wind_speed_{year}.csv'
    
    dataframes = {}
    
    # Read air temperature data
    if os.path.exists(temp_file):
        print(f"  Reading temperature data...")
        df_temp = pd.read_csv(temp_file)
        # Parse datetime and floor to day
        df_temp['Date'] = pd.to_datetime(df_temp['date'], errors='coerce').dt.floor('D')
        df_temp['Region'] = df_temp['station_name'].apply(map_station_to_region)
        
        # Group by Date (day) and Region, take mean
        df_temp = df_temp.groupby(['Date', 'Region']).agg({
            'reading_value': 'mean'
        }).reset_index()
        df_temp.columns = ['Date', 'Region', 'Temperature']
        df_temp['Temperature'] = df_temp['Temperature'].round(2)
        dataframes['temp'] = df_temp
        print(f"    - {len(df_temp)} daily temperature records processed")
    else:
        print(f"  Warning: {temp_file} not found")
    
    # Read humidity data
    if os.path.exists(humidity_file):
        print(f"  Reading humidity data...")
        df_humidity = pd.read_csv(humidity_file)
        df_humidity['Date'] = pd.to_datetime(df_humidity['date'], errors='coerce').dt.floor('D')
        df_humidity['Region'] = df_humidity['station_name'].apply(map_station_to_region)

        # Group by Date (day) and Region, take mean
        df_humidity = df_humidity.groupby(['Date', 'Region']).agg({
            'reading_value': 'mean'
        }).reset_index()
        df_humidity.columns = ['Date', 'Region', 'RelativeHumidity']
        df_humidity['RelativeHumidity'] = df_humidity['RelativeHumidity'].round(2)
        dataframes['humidity'] = df_humidity
        print(f"    - {len(df_humidity)} daily humidity records processed")
    else:
        print(f"  Warning: {humidity_file} not found")
    
    # Read wind speed data
    if os.path.exists(wind_file):
        print(f"  Reading wind speed data...")
        df_wind = pd.read_csv(wind_file)
        df_wind['Date'] = pd.to_datetime(df_wind['date'], errors='coerce').dt.floor('D')
        df_wind['Region'] = df_wind['station_name'].apply(map_station_to_region)

        # Group by Date (day) and Region, take mean
        df_wind = df_wind.groupby(['Date', 'Region']).agg({
            'reading_value': 'mean'
        }).reset_index()
        df_wind.columns = ['Date', 'Region', 'WindSpeed']
        df_wind['WindSpeed'] = df_wind['WindSpeed'].round(2)
        dataframes['wind'] = df_wind
        print(f"    - {len(df_wind)} daily wind speed records processed")
    else:
        print(f"  Warning: {wind_file} not found")
    
    # Merge all weather data
    if dataframes:
        # Start with first available dataframe
        df_combined = list(dataframes.values())[0].copy()
        
        # Merge with remaining dataframes
        for key, df in list(dataframes.items())[1:]:
            df_combined = pd.merge(df_combined, df, on=['Date', 'Region'], how='outer')
        
        # Add Country column
        df_combined.insert(0, 'Country', 'Singapore')
        
        # Format date as YYYY-MM-DD
        df_combined['Date'] = df_combined['Date'].dt.strftime('%Y-%m-%d')
        
        # Reorder columns
        columns_order = ['Country', 'Region', 'Date', 'Temperature', 'RelativeHumidity', 'WindSpeed']
        columns_order = [col for col in columns_order if col in df_combined.columns]
        df_combined = df_combined[columns_order]
        
        # Sort by date and region
        df_combined = df_combined.sort_values(['Date', 'Region']).reset_index(drop=True)
        
        # Round all numeric columns to 2 decimal places
        numeric_cols = df_combined.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            df_combined[col] = df_combined[col].round(2)
        
        # Save to CSV
        output_file = f'weather_{year}.csv'
        df_combined.to_csv(output_file, index=False)
        
        print(f"\n✓ Weather data for {year} saved to '{output_file}'")
        print(f"  Total records: {len(df_combined):,}")
        print(f"  Date range: {df_combined['Date'].min()} to {df_combined['Date'].max()}")
        print(f"  Regions: {sorted(df_combined['Region'].unique())}")
        
        return df_combined
    else:
        print(f"No weather data files found for {year}!")
        return None

# Process weather data for all years
print("\n" + "="*80)
print("CLEANING WEATHER DATA")
print("="*80)

weather_dataframes = {}
for year in range(2016, 2025):
    df = clean_weather_data_by_year(year)
    if df is not None:
        weather_dataframes[year] = df

# Display sample of cleaned weather data (latest year)
if weather_dataframes:
    latest_year = max(weather_dataframes.keys())
    print("\n" + "="*80)
    print(f"WEATHER DATA SAMPLE ({latest_year})")
    print("="*80)
    print("\nFirst 10 records:")
    print(weather_dataframes[latest_year].head(10))
    print("\nData types:")
    print(weather_dataframes[latest_year].dtypes)
    print("\nSummary statistics:")
    print(weather_dataframes[latest_year].describe())
    print("\nMissing values:")
    missing = weather_dataframes[latest_year].isnull().sum()
    missing_pct = (missing / len(weather_dataframes[latest_year]) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Percentage': missing_pct
    })
    print(missing_df[missing_df['Missing Count'] > 0])

# ============================================================================
# 3. SUMMARY AND VERIFICATION
# ============================================================================

print("\n" + "="*80)
print("DATA CLEANING SUMMARY")
print("="*80)

print("\n📊 POLLUTANT DATA:")
if os.path.exists('pollutant_data.csv'):
    df_check = pd.read_csv('pollutant_data.csv')
    print(f"  ✓ File: pollutant_data.csv")
    print(f"  ✓ Records: {len(df_check):,}")
    print(f"  ✓ Columns: {list(df_check.columns)}")
    print(f"  ✓ Date range: {df_check['Date'].min()} to {df_check['Date'].max()}")
    print(f"  ✓ Regions: {sorted(df_check['Region'].unique())}")
    print(f"  ✓ Missing values:")
    for col in df_check.columns:
        missing = df_check[col].isna().sum()
        if missing > 0:
            print(f"      {col}: {missing:,} ({missing/len(df_check)*100:.2f}%)")
else:
    print("  ✗ File not found")

print("\n🌤️  WEATHER DATA:")
for year in range(2016, 2025):
    filename = f'weather_{year}.csv'
    if os.path.exists(filename):
        df_check = pd.read_csv(filename)
        print(f"  ✓ {filename}:")
        print(f"      Records: {len(df_check):,}")
        print(f"      Columns: {list(df_check.columns)}")
        print(f"      Date range: {df_check['Date'].min()} to {df_check['Date'].max()}")
        missing_summary = []
        for col in df_check.select_dtypes(include=[np.number]).columns:
            missing = df_check[col].isna().sum()
            if missing > 0:
                missing_summary.append(f"{col}={missing}")
        if missing_summary:
            print(f"      Missing: {', '.join(missing_summary)}")

print("\n" + "="*80)
print("✓ Data cleaning completed successfully!")
print("="*80)

print("\n📋 OUTPUT FILES:")
print("  - pollutant_data.csv (single file, 2016-2024, daily aggregation)")
print("  - weather_2016.csv through weather_2024.csv (yearly files, daily aggregation)")
print("\n💡 Both datasets now have ONE RECORD PER REGION PER DAY")

Libraries imported successfully!

CLEANING POLLUTANT DATA
Processing pollutant data for 2016...
  - Processed 1914 daily records from 2016
Processing pollutant data for 2017...
  - Processed 2166 daily records from 2017
Processing pollutant data for 2018...
  - Processed 2190 daily records from 2018
Processing pollutant data for 2019...
  - Processed 2160 daily records from 2019
Processing pollutant data for 2020...
  - Processed 2172 daily records from 2020
Processing pollutant data for 2021...
  - Processed 2118 daily records from 2021
Processing pollutant data for 2022...
  - Processed 2184 daily records from 2022
Processing pollutant data for 2023...
  - Processed 2015 daily records from 2023
Processing pollutant data for 2024...
  - Processed 1830 daily records from 2024

✓ Pollutant data cleaned and saved to 'pollutant_data.csv'
  Total records: 18,749
  Date range: 2016-02-07 to 2024-12-31
  Regions: ['Central', 'East', 'National', 'North', 'South', 'West']

  Records per region